# Expected behaviour of TD(0) in the off-policy prediction

## Aka: what makes the deadly triad so deadly

Hint: it's all about linear algebra :)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import time as time

# 1. Off-policy TD(0) with Linear VFA

## 1.1 Using State Value functions

We consider the off-policy case, extended to linear function approximation.

Recall the semi-gradient TD update :
\begin{align*}
\delta_{t} &= R_{t+1} + \gamma \hat{v}(S_{t+1},\theta_{t}) - \hat{v}(S_{t},\theta_{t}) \\
\theta_{t+1} &= \theta_{t} + \alpha \rho_{t} \delta_{t} \nabla\hat{v}(S_{t},\theta_{t})
\end{align*}

The expected behaviour then writes:
\begin{align*}
E[\theta_{t+1}|\theta_{t}] &= \theta_{t} + \alpha E[\rho_{t} \delta_{t} \nabla\hat{v}(S_{t},\theta_{t}) | \theta_{t}]\\
&= \theta_{t} + \alpha E\left[\rho_{t} \big( R_{t+1}\phi_{t} - \phi_{t} (\phi_{t} - \gamma \phi_{t+1} )^{T} \theta_{t} \big) | \theta_{t} \right]\\
&=\theta_{t} + \alpha E\left[\rho_{t} R_{t+1}\phi_{t}\right] - \alpha E\left[\rho_{t} \phi_{t} (\phi_{t} - \gamma \phi_{t+1} )^{T}  \right] \theta_{t}\\
\end{align*}

Now, defining $A,b$ as follow:
\begin{align*}
A&=E\left[\rho_{t} \phi_{t} (\phi_{t} - \gamma \phi_{t+1} )^{T}  \right]\\
b&=E\left[\rho_{t} R_{t+1}\phi_{t}\right]
\end{align*}

If we expand these expectations, we can write them under the following matrix form. Actually, these equations are at the beginning of the 2015 paper on emphatic TD (off-policy section).
\begin{align*}
A&=\Phi^{T}D_{\mu}(I-\gamma P_{\pi})\Phi\\
b&=\Phi^{T}D_{\mu}r_{\pi}
\end{align*}

Let's sum up what we know so far :

* On-policy. $TD(0)$ converges to the TD fixed point, all is fine
\begin{align*}
A&=\Phi^{T}D_{\pi}(I-\gamma P_{\pi})\Phi\\
b&=\Phi^{T}D_{\pi}r_{\pi}
\end{align*}

* Off policy, no IS. Not having any IS means we're actually learning the value function of behaviour policy, not that of the target.
\begin{align*}
A&=\Phi^{T}D_{\mu}(I-\gamma P_{\mu})\Phi\\
b&=\Phi^{T}D_{\mu}r_{\mu}
\end{align*}

* Off-policy, IS. Introducing IS actually corrects part of the equation, but not all of it. In expectation, everything goes as if the transitions occured according to the target policy, but states were sampled according to another distribution (that of the behaviour policy). This corresponds to the case considered in section IX of [Tsitsiklis, Van Roy 1997], where they show that TD can diverge if states are not sampled according to the right distribution.
\begin{align*}
A&=\Phi^{T}D_{\mu}(I-\gamma P_{\pi})\Phi\\
b&=\Phi^{T}D_{\mu}r_{\pi}
\end{align*}


In the on-policy case, the convergence prrof relied on the fact that states are sampled from the stationnary distribution $d_{\pi}$ (see 9.4 in Sutton's book). It is no longer the case in an off-policy. As a consequence, $A$ is no longer guaranteed to be positive definite, and TD may diverge.

Indeeed, assuming $A\theta_{TD} = b$, recall the error behaves as $\theta_{t+1} - \theta_{TD} = \epsilon_{t+1} = (I-\alpha A) \epsilon_{t}$. Thus, if $A$ has at least one negative eigenvalue, then TD is bound to diverge.

### 1.1.1 An example
Let's illustrate this in Baird's famous counterexample (picture taken from Sutton's book):
<img src="baird_scheme.png">

In [79]:
#number of states
n=7

#discount factor
gamma=0.99
r_pi=np.zeros(n)

#define stationnary distribution under mu
mu=1.0/n * np.ones(n)
D_mu =np.diag(mu)
#define stationnary distribution under pi
pi=np.zeros(n)
pi[n-1]=1.0
D_pi=np.diag(pi)

#Compute transition probability matrices for target and behaviour policy
P_pi=np.zeros((7,7))
P_pi[:,6]=1

P_mu=np.zeros((7,7))
P_mu[:,:]=1.0/7.0


#Define features matrix
Phi = np.zeros((7,8))
for i in range(6):
    Phi[i,i]=2
    Phi[i,7]=1
Phi[6,6]=1
Phi[6,7]=2


I=np.eye(7)

#Compute matrix A
A=Phi.T.dot(D_mu).dot(I-gamma*P_pi).dot(Phi)

#by computing the column sums of the key matrix, we see whether it is positive definite
print 'Column sums of key matrix'
print np.ones(n).T.dot(D_mu).dot(I-gamma*P_pi)

#We TD is bound to diverge (unless the initial theta is really-well chosen) 
#if A has at least one negative eigenvalue
[eA,vA]=np.linalg.eig(A)
print '\nEigenvalues of A'
print eA

Column sums of key matrix
[ 0.14285714  0.14285714  0.14285714  0.14285714  0.14285714  0.14285714
 -0.84714286]

Eigenvalues of A
[  5.71428571e-01  -2.39250464e-01  -2.21781072e-02  -7.94718827e-17
   5.71428571e-01   5.71428571e-01   5.71428571e-01   5.71428571e-01]


### 1.1.2 Are we doomed, though ?

Not necessarily, as shows the last line of the proof (still 9.4 in Sutton's book). We need the column sum of the key matrix to be positive. This ought to be the case, assuming $\mu$ is close enough to $\pi$.

The column sums of the key matrix write as :
\begin{align*}
1^{T} D_{\mu} (I-\gamma P_{\pi}) &= d_{\mu}^{T} (I-\gamma P_{\pi})\\
&= d_{\mu}^{T} - \gamma d_{\mu}^{T} P_{\pi}
\end{align*}

Whether we consider the behaviour policy or the target policy, the induced markov chain (with transition matrix $P_{\pi}$ or $P_{\mu}$) is irreductible (otherwise, we may restrict to a smaller set of states) and ergodic (otherwise, some states are not visited infinitely many times during learning). Therefore, the associated stationary distribution is unique. 

As $\mu \rightarrow \pi$, we have $P_{\pi} \rightarrow P_{\mu}$.  We also know that $d_{\pi}$ (resp $d_{\mu}$) is a left-eigenvector of $P_{\pi}$ (resp $P_{\mu}$) associated to left-eigenvalue $1$. One can show that there exists a continous mapping from $\mu$ to $d_{\mu}$.

Using the triangular inequality :
\begin{align*}
d_{\mu}-d_{\pi} &= d_{\mu}- \nu^{T} P_{\mu}^{t} + \nu^{T} P_{\mu}^{t} - \nu^{T} P_{\pi}^{t} + \nu^{T} P_{\pi}^{t}-d_{\pi} \\
\|d_{\mu}-d_{\pi}\| &\leq \| d_{\mu}- \nu^{T} P_{\mu}^{t} \| + \| \nu^{T} P_{\mu}^{t} - \nu^{T} P_{\pi}^{t} \| + \|\nu^{T} P_{\pi}^{t}-d_{\pi}\|
\end{align*}

For instance, denote $\nu$ the distribution over initial states. Then :
\begin{align*}
d_{\mu}^{T} &= \lim_{t \rightarrow +\infty} \nu^{T} P_{\mu}^{t}\\
d_{\pi}^{T} &= \lim_{t \rightarrow +\infty} \nu^{T} P_{\pi}^{t}
\end{align*}

So, for some $\eta>0$, and $t>T$, we have:
\begin{align*}
\| d_{\mu}^{T} - \nu^{T} P_{\mu}^{t}\| \leq \eta \\
\| d_{\pi}^{T} - \nu^{T} P_{\pi}^{t} \| \leq \eta
\end{align*}

Finally, when $\mu$ is close to $\pi$, $d_{\mu}^{T} - \gamma d_{\mu}^{T} P_{\pi}$ is close to $d_{\pi}^{T} - \gamma d_{\pi}^{T} P_{\pi}$ which coefficients are all positive.

As a conclusion : just like there always exist a behaviour policy that will make off-policy TD with linear approxiamtion diverge, there probably exists at least one that ensures it converges.

### 1.2.3 Why is TD guaranted to converge in the tabular case ?

I believe the answer is in the projector operator $\Pi$. Recall that :
$$
\Pi = \Phi (\Phi^{T} D \Phi)^{-1} \Phi^{T} D
$$

The problem with off-policy is that, if $D$ is not the stationnary distribution of the Markov Chain, then the TD operator may not be a contraction with respect to $D$ (see [Tsitsiklis 97]). However, in the tabular case, $\Phi$ is the identity matrix, which yields immediately $\Pi=I$, no matter which distribution we consider. As a consequence, $\Pi \cdot TD$ is a contraction (because TD is), regardless of the behaviour policy (we still require coverage).

## 1.2 Using State-Action functions (aka, Q-learning)

We now consider state-action values. The corresponding algorithm is Expected Sarsa, aka Q-learning when the policy is greedy. Recall the semi-gradient algorithm :
\begin{align*}
\delta_{t} &= R_{t+1} + \gamma \sum_{a} \pi(a|S_{t+1}) \hat{q}(S_{t+1},a,\theta_{t}) - \hat{q}(S_{t},a,\theta_{t}) \\
\theta_{t+1} &= \theta_{t} + \alpha \rho_{t} \delta_{t} \nabla\hat{q}(S_{t},A_{t},\theta_{t})
\end{align*}

We use linear function approximation, ie : $\hat{q}(s,a) = \theta^{T}\phi(s,a)$. We then have :
$$
\theta_{t+1} = \theta_{t} +\alpha \left[ R_{t+1} \Phi(S_{t},A_{t}) - \Phi(S_{t},A_{t}) \big( \Phi(S_{t},A_{t}) - \gamma \sum_{a} \pi(a|S_{t+1}) \Phi(S_{t+1},a)\big)^{T} \theta_{t} \right]
$$

In expectation, updates thus write as (to ease the reading, I voluntarily skip a few lines of calculus here):
\begin{align*}
E[\theta_{t+1}|\theta_{t}] &= \theta_{t} +\alpha E\left[ R_{t+1} \Phi(S_{t},A_{t})\right] - E\left[ \Phi(S_{t},A_{t}) \big( \Phi(S_{t},A_{t}) - \gamma \sum_{a} \pi(a|S_{t+1}) \Phi(S_{t+1},a)\big)^{T}  \right] \theta_{t}\\
&=\theta_{t} + \alpha(b-A \theta_{t})
\end{align*}

where (all matrices and vectors are now indexed by state-action pairs) :
\begin{align*}
A&=\Phi^{T}D_{\mu}(I-\gamma P_{\pi})\Phi\\
b&=\Phi^{T}D_{\mu}r
\end{align*}


The main result here, is that these expression are obtained without using importance sampling. This is due to the fact that Q-learning samples directly from state-action together. Therefore, the transition to the next state $S_{t+1}$ does not depend on the behaviour policy. Besides, the expectation in the target is taken explicitely with respect to $\pi$. Finally, notice the reward vector $r$ also does not depend on the behaviour policy (nor the target).

We have the exact same structure as we had before. Ergo, the conclusions are the same.

## 2. What's next ?

From here, I believe emphatic TD is the natural way to go. We've seen that importance sampling corrects some things in the off policy case, but not enough. Emphatic TD seems to be able to cope with the fact that states are sampled from a different distribution.

That's what state weighting is about.